   # TP53: Effects of Mutation

<b>Standard imports for playing with and plotting data frames.</b>

In [3]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

<b>Import CPTAC data</b>

In [4]:
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******


In [5]:
somatic_mutations = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

### List of proteins that are affected by TP53 (somewhere along the pathway)

In [10]:
protList = ['BAX', 'FAS', 'BCL2', 'CREBBP', 'CDK2', 'MDM2', 'CDKN2A', 'CDKN1A', 'ATM', 'MDM4']

### Proteome abundance

In [12]:
gene = 'TP53'
sigList = [];

pcutoff = 0.05/len(protList)

genedf = somatic_mutations[gene].to_frame()
for protein in protList:
    if protein in proteomics.columns:
        proteindf = proteomics[protein].to_frame()
        proteindfName = protein + " protein"
        proteindf.columns = [proteindfName]
        cross = genedf.add(proteindf, fill_value=0).dropna(axis=0)
        mutated = cross.loc[cross[gene] == 1.0]
        wt = cross.loc[cross[gene] == 0.0]
        ttest = scipy.stats.ttest_ind(mutated[proteindfName], wt[proteindfName])
        if ttest[1] <= pcutoff:
            sigList.append(protein)
            print("Test for " + protein + ": ")
            print(ttest)

Test for FAS: 
Ttest_indResult(statistic=-5.5103825350041875, pvalue=2.894104669939677e-07)
Test for CDK2: 
Ttest_indResult(statistic=3.5157649029196443, pvalue=0.0006658078140619433)
Test for CDKN2A: 
Ttest_indResult(statistic=4.344971931273746, pvalue=3.583610475413602e-05)
Test for CDKN1A: 
Ttest_indResult(statistic=-3.2487489884354845, pvalue=0.0016136342251464122)


### List of significantly affected proteins

In [13]:
print(sigList)

['FAS', 'CDK2', 'CDKN2A', 'CDKN1A']


### Phosphoproteome abundance of pathway proteins

In [14]:
gene = 'TP53'
genedf = somatic_mutations[gene].to_frame()
sigPhosResults = [];

for protein in protList:
    sites = phos.filter(regex=protein)
    genedf = genedf.add(sites, fill_value=0)

mutated = genedf.loc[genedf[gene] == 1.0]
wt = genedf.loc[genedf[gene] == 0.0]

pcutoff = 0.05 / len(genedf.columns)

for loc in genedf.columns:
    if not loc == gene:
        mutsitedf = mutated[[gene, loc]].dropna()
        wtsitedf = wt[[gene, loc]].dropna()
        ttest = scipy.stats.ttest_ind(mutsitedf[loc], wtsitedf[loc])
        if(ttest[1] <= pcutoff):
            sigPhosResults.append(loc)
            print('Results for ' + loc + ': ')
            print(ttest)
 

Results for BCL2L12-S273: 
Ttest_indResult(statistic=3.6925070055844387, pvalue=0.0004158661622105229)
Results for CREBBP-T974: 
Ttest_indResult(statistic=-3.9777257442589176, pvalue=0.0002018327186438462)


### List of significantly affected phosphorylation sites

In [15]:
print(sigPhosResults)

['BCL2L12-S273', 'CREBBP-T974']
